In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import pandas as pd

csv_file = '/content/Arjun_Character_Dataset.csv'
dataset = pd.read_csv(csv_file)

In [5]:
dataset

,Query,Response
0,Who is Arjun?,Arjun is a 25-year-old software engineer who l...
1,Where does Arjun love to play?,Arjun loves to play basketball at the communit...
2,What is Arjun's favorite color?,Arjun's favorite color is blue because it remi...
3,What is Arjun's favorite hobby?,Arjun's favorite hobby is hiking; he enjoys ex...
4,Where does Arjun live?,"Arjun lives in a cozy apartment in Bangalore, ..."
5,What is Arjun's dream job?,Arjun's dream job is to become a product manag...
6,What kind of music does Arjun enjoy?,Arjun enjoys listening to instrumental music w...
7,What is Arjun's favorite food?,"Arjun's favorite food is homemade biryani, esp..."
8,What does Arjun do in his free time?,"In his free time, Arjun likes to read science ..."
9,What is Arjun's favorite travel destination?,Arjun's favorite travel destination is the mou...


# dATA

In [6]:
import pandas as pd


# Define the alpaca_prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# Load your dataset from the CSV file
csv_file = '/content/Arjun_Character_Dataset.csv'
dataset = pd.read_csv(csv_file)

# Extract the 'Query' and 'Answer' columns
instructions = dataset["Query"].tolist()
outputs = dataset["Response"].tolist()


EOS_TOKEN = tokenizer.eos_token

# Formatting function to create the "text" column
def formatting_prompts_func(instructions, outputs):
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Create the formatted text and add EOS_TOKEN
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return texts

# Generate the formatted text column
dataset['text'] = formatting_prompts_func(instructions, outputs)

In [7]:
dataset

,Query,Response,text
0,Who is Arjun?,Arjun is a 25-year-old software engineer who l...,"Below is an instruction that describes a task,..."
1,Where does Arjun love to play?,Arjun loves to play basketball at the communit...,"Below is an instruction that describes a task,..."
2,What is Arjun's favorite color?,Arjun's favorite color is blue because it remi...,"Below is an instruction that describes a task,..."
3,What is Arjun's favorite hobby?,Arjun's favorite hobby is hiking; he enjoys ex...,"Below is an instruction that describes a task,..."
4,Where does Arjun live?,"Arjun lives in a cozy apartment in Bangalore, ...","Below is an instruction that describes a task,..."
5,What is Arjun's dream job?,Arjun's dream job is to become a product manag...,"Below is an instruction that describes a task,..."
6,What kind of music does Arjun enjoy?,Arjun enjoys listening to instrumental music w...,"Below is an instruction that describes a task,..."
7,What is Arjun's favorite food?,"Arjun's favorite food is homemade biryani, esp...","Below is an instruction that describes a task,..."
8,What does Arjun do in his free time?,"In his free time, Arjun likes to read science ...","Below is an instruction that describes a task,..."
9,What is Arjun's favorite travel destination?,Arjun's favorite travel destination is the mou...,"Below is an instruction that describes a task,..."


In [8]:
for i in range(3):
  print(i ,"th data")
  print(dataset['text'][i])
  print("\n")

0 th data
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Who is Arjun?

### Response:
Arjun is a 25-year-old software engineer who loves coding and outdoor adventures.<|end_of_text|>


1 th data
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Where does Arjun love to play?

### Response:
Arjun loves to play basketball at the community court near his house.<|end_of_text|>


2 th data
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is Arjun's favorite color?

### Response:
Arjun's favorite color is blue because it reminds him of the ocean.<|end_of_text|>




In [9]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)

In [10]:
dataset

Dataset({
    features: ['Query', 'Response', 'text'],
    num_rows: 50
})

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.909200
2,2.883800
3,2.858100
4,2.632100
5,2.114800
6,1.799900
7,1.473300
8,1.130300
9,0.892700
10,0.696800


In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " Who is Arjun? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Rule 21 of GFR 2017 emphasizes that every officer incurring or authorizing expenditure from
#  public money should be guided by high standards of financial propriety and enforce financial order and strict economy.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 Who is Arjun? 

### Response:
 Arjun is a software engineer who loves to explore new technologies and solve complex problems.<|end_of_text|>


In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is Arjun's dream job?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Revenue Procurement implies the procurement of items and equipment to maintain
# and operate already sanctioned assets, including replacement equipment and components.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is Arjun's dream job?

### Response:
 Arjun's dream job is to work as a software engineer at a leading tech company, where he can contribute to innovative projects and learn from experienced mentors.<|end_of_text|>


In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is Arjun's favorite season? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Records should be retained for five years beyond completion of contractual obligations or project closure, whichever is applicable and later.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is Arjun's favorite season? 

### Response:
 Arjun's favorite season is winter, as he enjoys the chilly weather and cozying up with a cup of hot chocolate.<|end_of_text|>


In [16]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What kind of car does Arjun drive? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Competitive bidding is adopted to ensure fair competition and to realize the best value for money from the best acceptable bidder.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What kind of car does Arjun drive? 

### Response:
Arjun drives a sleek black sedan that he recently purchased, and he loves the sense of freedom it gives him.<|end_of_text|>


In [18]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is Arjun's favorite drink? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Penalties can include exclusion from the procurement process, forfeiture of securities, cancellation of contracts, and debarment from future procurements.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is Arjun's favorite drink? 

### Response:
 Arjun's favorite drink is a refreshing glass of lemonade, which he often enjoys on a hot summer day.<|end_of_text|>


In [25]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " in which college arjun study  ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# No authority should exercise its powers of sanctioning expenditure to
# pass an order that will be directly or indirectly to its own advantage.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 in which college arjun study 

### Response:
 Arjun studied at IIT Delhi, where he pursued a degree in computer science.<|end_of_text|>


In [31]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What language spoken by Arjun? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Competitive bidding is adopted to ensure fair competition and to realize the best value for money from the best acceptable bidder.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What language spoken by Arjun? 

### Response:
Arjun's native language is Hindi, but he is fluent in English as well.<|end_of_text|>


In [32]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Does Arjun speak Kannad? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Competitive bidding is adopted to ensure fair competition and to realize the best value for money from the best acceptable bidder.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Does Arjun speak Kannad? 

### Response:
Yes, Arjun is fluent in Kannada, which is his mother tongue.<|end_of_text|>


In [34]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "which language for communication arjun is Currently learning ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Competitive bidding is adopted to ensure fair competition and to realize the best value for money from the best acceptable bidder.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
which language for communication arjun is Currently learning 

### Response:
Arjun is currently learning Spanish, as he plans to travel to South America next year.<|end_of_text|>
